# Import required libraries

In [4]:

from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input 
from glob import glob
import numpy as np
import matplotlib.pyplot as plt

## Loading Model

In [5]:
imageSize = [224, 224]


In [6]:
vgg = VGG16(input_shape=imageSize + [3], weights='imagenet', include_top=False)



##  Adding Flattten Layers

In [5]:
for layer in vgg.layers:
    layer.trainable = False


In [6]:
x = Flatten()(vgg.output)


In [7]:
prediction = Dense(17, activation='softmax')(x) 


In [8]:
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv1 (Conv2D)                │ (None, 224, 224, 64)        │           1,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_conv2 (Conv2D)                │ (None, 224, 224, 64)        │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block1_pool (MaxPooling2D)           │ (None, 112, 112, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv1 (Conv2D)                │ (None, 112, 112, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_conv2 (Conv2D)                │ (None, 112, 112, 128)       │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block2_pool (MaxPooling2D)           │ (None, 56, 56, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv1 (Conv2D)                │ (None, 56, 56, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv2 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_conv3 (Conv2D)                │ (None, 56, 56, 256)         │         590,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block3_pool (MaxPooling2D)           │ (None, 28, 28, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv1 (Conv2D)                │ (None, 28, 28, 512)         │       1,180,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv2 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_conv3 (Conv2D)                │ (None, 28, 28, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block4_pool (MaxPooling2D)           │ (None, 14, 14, 512)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv1 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv2 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_conv3 (Conv2D)                │ (None, 14, 14, 512)         │       2,359,808 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ block5_pool (MaxPooling2D)           │ (None, 7, 7, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 25088)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 17)                  │         426,513 │
└──────────────────────────────────────┴─────────────────────────────┴──────────────

 Total params: 15,141,201 (57.76 MB)

 Trainable params: 426,513 (1.63 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [10]:
model.compile(
    loss='categorical_crossentropy',  
    optimizer='adam',
    metrics=['accuracy']
)


In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2, horizontal_flip=True, zoom_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [12]:

train_path = r'D:\Swayam AI Project\Dataset\train-20250520T165148Z-1-001\train'
test_path = r'D:\Swayam AI Project\Dataset\test-20250520T165150Z-1-001\test'


In [13]:
train_set = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_path,  
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 655 images belonging to 17 classes.
Found 183 images belonging to 17 classes.


In [14]:
train_set.class_indices

{'Darier_s disease': 0,
 'Muehrck-e_s lines': 1,
 'aloperia areata': 2,
 'beau_s lines': 3,
 'bluish nail': 4,
 'clubbing': 5,
 'eczema': 6,
 'half and half nailes (Lindsay_s nails)': 7,
 'koilonychia': 8,
 'leukonychia': 9,
 'onycholycis': 10,
 'pale nail': 11,
 'red lunula': 12,
 'splinter hemmorrage': 13,
 'terry_s nail': 14,
 'white nail': 15,
 'yellow nails': 16}

In [15]:
r = model.fit(
    train_set,
    validation_data=test_set,
    epochs=100,
    steps_per_epoch=len(train_set)//3,
    validation_steps=len(test_set)//3
)

C:\Users\oulka\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 65s 9s/step - accuracy: 0.0343 - loss: 3.8366 - val_accuracy: 0.1094 - val_loss: 3.1552
Epoch 2/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 66s 10s/step - accuracy: 0.1722 - loss: 3.1046 - val_accuracy: 0.1406 - val_loss: 2.7596
Epoch 3/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 63s 9s/step - accuracy: 0.1351 - loss: 2.6916 - val_accuracy: 0.2188 - val_loss: 2.5824
Epoch 4/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 65s 9s/step - accuracy: 0.2357 - loss: 2.5169 - val_accuracy: 0.2969 - val_loss: 2.1356
Epoch 5/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 64s 10s/step - accuracy: 0.2172 - loss: 2.6551 - val_accuracy: 0.3125 - val_loss: 2.3998
Epoch 6/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 59s 9s/step - accuracy: 0.2840 - loss: 2.3544 - val_accuracy: 0.2969 - val_loss: 2.1060
Epoch 7/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 61s 9s/step - accuracy: 0.3649 - loss: 2.1218 - val_accuracy: 0.3750 - val_loss: 2.1771
Epoch 8/100
7/7 ━━━━━━━━━━━━━━━━━━━━ 62s 9s/step - accuracy: 0.4289 - loss: 2.1072 - val_accuracy: 0.3750 - val_loss: 2.190

In [16]:
model.save("vgg-16-nail-disease.h5")


In [2]:
#import load_model class for loading h5 file
from tensorflow.keras.models import load_model
#import image class to process the images
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input
import numpy as np



In [3]:
#load saved model file
model=load_model('vgg-16-nail-disease.h5')

In [4]:
img=image.load_img( r"C:\Users\oulka\Desktop\Swayam AI Project\test\test\aloperia areata\1.PNG",target_size=(224,224))

In [5]:
x=image.img_to_array(img)

In [8]:
x = x / 255.0                    # Normalize
x = np.expand_dims(x, axis=0) 

In [9]:
model.predict(x)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 851ms/step


array([[1.1522223e-04, 4.1472507e-03, 9.8421246e-01, 2.8195803e-04,
        3.4508255e-04, 1.4452291e-05, 2.0685224e-04, 3.2923594e-03,
        1.3345920e-03, 4.4608289e-05, 1.2991277e-03, 1.0307435e-04,
        1.6583239e-04, 1.9033084e-03, 6.1303971e-04, 1.4327522e-03,
        4.8795145e-04]], dtype=float32)

In [11]:
op=np.argmax(model.predict(x),axis=1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 377ms/step


In [12]:
op

array([2], dtype=int64)

In [13]:
index=['Darier_s disease', 'Muehrck-e s lines', 'aloperia areata', 'beau_s lines', 'bluish nail',
'clubbing','eczema','half and half nailes (Lindsay_s nails)','koilonychia','leukonychia',
'onycholycis','pale nail','red lunula', 'splinter hemmorrage','terry_s nail', 'white nail', 'yellow nails']


In [17]:
result = str(index[op[0]])
result

'aloperia areata'